In [1]:
# Change directory to the root of the project
import os 
os.chdir('..')
os.chdir('..')
os.chdir('..')
print(f"Working directory: {os.getcwd()}")

Working directory: /Users/eohjelle/Documents/2025-dots-and-boxes/dots-and-boxes


In [2]:
from applications.tic_tac_toe.models.experimental_transformer import ExperimentalTransformerInitParams
import torch

# Initialize parameters

## Model parameters
model_type = 'experimental_transformer'
model_params: ExperimentalTransformerInitParams = {
    'embed_dim': 32,
    'num_heads': 4
}
device = torch.device('mps')
model_name = 'tic_tac_toe_experimental_transformer'

## Initialize new model
load_model = None
load_model_params = {}


## Optimizer parameters
optimizer_type = 'adam'
optimizer_params = {
    'lr': 1e-2,
    'betas': (0.9, 0.999),
    'eps': 1e-8,
    'weight_decay': 1e-4,
    'amsgrad': False
}

## Learning scheduler parameters
lr_scheduler_type = 'plateau'
lr_scheduler_params = {
    'factor': 0.5,
    'patience': 25,
    'cooldown': 50,
    'min_lr': 1e-6
}

## Training parameters
training_method = 'supervised'
trainer_params = {}
training_params = {
    'epochs': 1000,
    'batch_size': 256,
    'eval_freq': 25,
    'checkpoint_freq': 50,
    'mask_illegal_moves': False,
    'mask_value': -20.0, # Doesn't matter when mask_illegal_moves is False
    'checkpoint_dir': 'checkpoints',
    'start_at': 1
}

## Load replay buffer from wandb
load_replay_buffer = 'from_wandb'
load_replay_buffer_params = {
    'project': 'AlphaZero-TicTacToe',
    'artifact_name': f'tic_tac_toe_TokenizedTensorMapping_training_data',
    'artifact_version': 'latest'
}


In [3]:
# Initialize wandb run
import wandb

run_name = 'Experimental Transformer 15'
notes = 'Same as v13 expect for: no norm1 and with L2 normalization of k, v instead of L1.'

config = {
    'model_type': model_type,
    'model_params': model_params,
    'optimizer_type': optimizer_type,
    'optimizer_params': optimizer_params,
    'lr_scheduler_type': lr_scheduler_type,
    'lr_scheduler_params': lr_scheduler_params,
    'training_method': training_method,
    'trainer_params': trainer_params,
    'training_params': training_params
}

run = wandb.init(
    project='AlphaZero-TicTacToe',
    name=run_name,
    config=config,
    notes=notes
)

wandb: Currently logged in as: eohjelle (eigenway) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


In [4]:
# Perform training

from applications.tic_tac_toe.train import train

model_interface = train(
    model_type=model_type,
    model_params=model_params,
    device=device,
    model_name=model_name,
    optimizer_type=optimizer_type,
    optimizer_params=optimizer_params,
    lr_scheduler_type=lr_scheduler_type,
    lr_scheduler_params=lr_scheduler_params,
    training_method=training_method,
    trainer_params=trainer_params,
    training_params=training_params,
    load_model=load_model,
    load_model_params=load_model_params,
    load_replay_buffer=load_replay_buffer,
    load_replay_buffer_params=load_replay_buffer_params,
    wandb_run=run
)

wandb: logging graph, to disable use `wandb.watch(log_graph=False)`
wandb:   1 of 1 files downloaded.  
/Users/eohjelle/Documents/2025-dots-and-boxes/dots-and-boxes/core/data_structures.py:94: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue

Epoch 1/200, Train Loss: 1.9790, Val Loss: 1.6763, Learning rate: 0.010000
Epoch 2/200, Train Loss: 1.6393, Val Loss: 1.6817, Learning rate: 0.010000
Epoch 3/200, Train Loss: 1.5869, Val Loss: 1.6270, Learning rate: 0.010000
Epoch 4/200, Train Loss: 1.6103, Val Loss: 1.5910, Learning rate: 0.010000
Epoch 5/200, Train Loss: 1.5826, Val Loss: 1.6029, Learning rate: 0.010000
Epoch 6/200, Train Loss: 1.5696, Val Loss: 1.6162, Learning rate: 0.010000
Epoch 7/200, Train Loss: 1.5846, Val Loss: 1.5924, Learning rate: 0.010000
Epoch 8/200, Train Loss: 1.5632, Val Loss: 1.5347, Learning rate: 0.010000
Epoch 9/200, Train Loss: 1.5217, Val Loss: 1.5668, Learning rate: 0.010000
Epoch 10/200, Train Loss: 1.5028, Val Loss: 1.4767, Learning rate: 0.010000
Epoch 11/200, Train Loss: 1.4162, Val Loss: 1.4227, Learning rate: 0.010000
Epoch 12/200, Train Loss: 1.3698, Val Loss: 1.4149, Learning rate: 0.010000
Epoch 13/200, Train Loss: 1.3157, Val Loss: 1.2286, Learning rate: 0.010000
Epoch 14/200, Train L

In [5]:
run.finish()

MCTS_draw_rate,▁▅██▇█▇▆
MCTS_loss_rate,█▂▁▁▂▁▂▁
MCTS_score,▁▇▇▇▆▇▇█
MCTS_win_rate,▁▇▂▃▂▁▃█
Minimax_draw_rate,▁▇████▇█
Minimax_loss_rate,█▂▁▁▁▁▂▁
Minimax_score,▁▇████▇█
Minimax_win_rate,▁▁▁▁▁▁▁▁
RandomAgent_draw_rate,▁▄▅▆█▅▁▅
RandomAgent_loss_rate,█▁▁▁▂▁▄▁
RandomAgent_score,▁█▇▆▂▇▇▇


In [6]:
def print_model_parameters(model):
    """
    Print the total number of parameters in a PyTorch model,
    with a breakdown of trainable vs non-trainable parameters.
    
    Args:
        model: PyTorch model
    """
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    non_trainable_params = total_params - trainable_params
    
    print(f"Total parameters: {total_params:,}")
    print(f"Trainable parameters: {trainable_params:,} ({trainable_params/total_params:.2%})")
    print(f"Non-trainable parameters: {non_trainable_params:,} ({non_trainable_params/total_params:.2%})")
    
    # Optional: Print parameters by layer
    print("\nParameters by layer:")
    for name, param in model.named_parameters():
        print(f"{name}: {param.numel():,} parameters")

# Example usage
print_model_parameters(model_interface.model)

Total parameters: 13,798
Trainable parameters: 13,798 (100.00%)
Non-trainable parameters: 0 (0.00%)

Parameters by layer:
pos_embedding: 288 parameters
input_embedding.weight: 96 parameters
norm1.weight: 32 parameters
norm1.bias: 32 parameters
attn.mask: 324 parameters
attn.q_emb.weight: 1,024 parameters
attn.q_emb.bias: 32 parameters
attn.k_emb.weight: 1,024 parameters
attn.k_emb.bias: 32 parameters
attn.v_emb.weight: 1,024 parameters
attn.v_emb.bias: 32 parameters
attn.out_emb.weight: 1,024 parameters
attn.out_emb.bias: 32 parameters
norm2.weight: 32 parameters
norm2.bias: 32 parameters
mlp.0.weight: 4,096 parameters
mlp.0.bias: 128 parameters
mlp.2.weight: 4,096 parameters
mlp.2.bias: 32 parameters
norm3.weight: 32 parameters
norm3.bias: 32 parameters
policy_head.weight: 32 parameters
policy_head.bias: 1 parameters
value_head.weight: 288 parameters
value_head.bias: 1 parameters
